# Term1 Sprint12 授業課題 
## コーディング課題：畳み込みニューラルネットワーク(CNN)スクラッチ1

NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していく。

Sprint11で作成したディープニューラルネットワークのクラスを拡張する形でCNNを作成する。  
まず、Sprint12で1次元畳み込み層を作成し、畳み込みの基礎を理解することを目指す。  
そして、Sprint13で一般的に画像に対して使われる2次元畳み込み層とプーリング層を作成する。

**1次元畳み込み層**  
畳み込みニューラルネットワークは画像に対して使われる2次元畳み込みが代表的だが、理解を容易にするためにまずは1次元畳み込みを実装する。  
1次元畳み込みは系列データで使われることが多い。  
畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまでがフレームワークで一般的に用意されている。

**データセットの用意**  
引き続きMNISTデータセットを使用する。  
1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力する。

**CNN分類器クラスの作成**  
1次元畳み込みニューラルネットワークモデルのクラスScratch1dCNNClassifierを作成する。  
Sprint11で作成したScratchDeepNeuralNetrowkClassifierを元にする。

## 1. チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成する。  
基本構造はsprint11で作成したFCクラスと同じになる。  
なお、重みの初期化に関するクラスは必要に応じて作り変える。Xavierの初期値などを使う点は全結合層と同様。

ここではパディングは考えず、ストライドも1に固定する。  
また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応する。  
この部分の拡張はアドバンス課題とする。

フォワードプロパゲーションの数式は以下のようになる。
$$a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b$$
$a_i$ : 出力される配列のi番目の値  
$F$ : フィルタのサイズ  
$x_{(i+s)}$ : 入力の配列の(i+s)番目の値  
$w_s$ : 重みの配列のs番目の値
$b$ : バイアス項

全てスカラーとなる。

次に更新式は以下のようになる。ここがAdaGradなどに置き換えられる点は全結合層と同様。
$$w_s^{\prime} = w_s - \alpha \frac{\partial L}{\partial w_s}$$

$$b^{\prime} = b - \alpha \frac{\partial L}{\partial b}$$

$\alpha$ : 学習率
$\frac{\partial L}{\partial w_s}$ : $w_s$に関する損失$L$の勾配
$\frac{\partial L}{\partial b}$ : $b$に関する損失$L$の勾配

勾配$\frac{\partial L}{\partial w_s}$や$\frac{\partial L}{\partial b}$を求めるためのバックプロパゲーションの数式は以下の通り。
$$\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}x_{(i+s)}$$

$$\frac{\partial L}{\partial b} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}$$

$\frac{\partial L}{\partial a_i}$ : 勾配の配列の$i$番目の値
$N_{out}$ : 出力のサイズ

前の層に流す誤差の数式は以下の通り。
$$\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1} \frac{\partial L}{\partial a_{(j-s)}}w_s$$

$\frac{\partial L}{\partial x_j}$ : 前の層に流す誤差の配列の$j$番目の値

ただし、$j−s<0$または$j−s>N_{out}−1$のとき$\frac{\partial L}{\partial a_{(j-s)}}=0$となる。

全結合層との大きな違いは、重みが複数の特徴量に対して共有されていること。  
この場合は共有されている分の誤差を全て足すことで勾配を求める。  
計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになる。

## 2. 1次元畳み込み後の出力サイズの計算
畳み込みを行うと特徴量の数が変化する。どのように変化するかは以下の数式から求められる。  
パディングやストライドも含めている。この計算を行う関数を作成する。

$$N_{out} =  \frac{N_{in}+2P-F}{S} + 1$$

$N_{out}$ : 出力のサイズ（特徴量の数）  
$N_{in}$ : 入力のサイズ（特徴量の数）  
$P$ : ある方向へのパディングの数  
$F$ : フィルタのサイズ  
$S$ : ストライドのサイズ

## 3. 小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認する。  
入力$x$、重み$w$、バイアス$b$を次のようにする。


In [ ]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

フォワードプロパゲーションをすると出力は次のようになる。

In [ ]:
a = np.array([35, 50])

次にバックプロパゲーションを考えます。誤差は次のようであったとする。

In [ ]:
delta_a = np.array([10, 20])

バックプロパゲーションをすると次のような値になる。

In [ ]:
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])

**実装上の工夫**  
畳み込みを実装する場合は、まずはfor文を重ねていく形で構わない。  
しかし、できるだけ計算は効率化させたいため、以下の式を一度に計算する方法を考える。
$$a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b$$

バイアス項は単純な足し算のため、重みの部分を見る。  
$$\sum_{s=0}^{F-1}x_{(i+s)}w_s$$

これは、xの一部を取り出した配列とwの配列の内積である。  
具体的な状況を考えると、以下のようなコードで計算できる。  
この例では流れを分かりやすくするために、各要素同士でアダマール積を計算してから合計を計算している。  
これは結果的に内積と同様となる。

In [ ]:
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])

a = np.empty((2, 3))

indexes0 = np.array([0, 1, 2]).astype(np.int)
indexes1 = np.array([1, 2, 3]).astype(np.int)

a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である

a = a.sum(axis=1)

ndarrayは配列を使ったインデックス指定ができることを利用した方法である。  
また、二次元配列を使えば一次元配列から二次元配列が取り出せる。

In [ ]:
x = np.array([1, 2, 3, 4])
indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)

print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])

このこととブロードキャストなどをうまく組み合わせることで、一度にまとめて計算することも可能。

畳み込みの計算方法に正解はないので、自分なりに効率化していく。

**参考**

以下のページのInteger array indexingの部分がこの方法についての記述である。

[Indexing — NumPy v1.15 Manual](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html "Indexing — NumPy v1.15 Manual")



## 4. チャンネル数を限定しない1次元畳み込み層クラスの作成
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成する。

紙やホワイトボードを使い計算グラフを書きながら考える。

例えば以下のような$x$, $w$, $b$があった場合は、

In [ ]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

出力は次のようになる。

In [ ]:
a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である。

入力が2チャンネル、出力が3チャンネルの例となる。  
計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみる。  
計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はない。

**補足**

チャンネル数を加える場合、配列をどういう順番にするかという問題がある。  
(バッチサイズ、チャンネル数、特徴量数)または(バッチサイズ、特徴量数、チャンネル数)が一般的で、ライブラリによって順番は異なる。（切り替えて使用できるものもある）

今回のスクラッチでは自身の実装上どちらが効率的かを考えて選ぶ。  
上記の例ではバッチサイズは考えておらず、(チャンネル数、特徴量数)となっている。

## 5. 学習・推定
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えて学習と推定を行う。  
出力層だけは全結合層をそのまま使う。

チャンネルが複数ある状態では全結合層への入力は行えない。  
その段階でのチャンネルは1になるようにするか、平滑化を行う。  
平滑化はNumPyのreshapeが使用できる。

[numpy.reshape — NumPy v1.15 Manual](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.reshape.html "numpy.reshape — NumPy v1.15 Manual")

画像に対しての1次元畳み込みは実用上は行わないことであるため、精度は問わない。

## 6. パディングの実装（アドバンス課題）
畳み込み層にパディングを加える。  
1次元配列の場合、前後にn個特徴量を増やせるようにする。

最も単純なパディングは全て0で埋めるゼロパディングであり、CNNでは一般的。  
他に端の値を繰り返す方法などもある。

フレームワークによっては、元の入力のサイズを保つようにという指定をすることができる。  
この機能も持たせておくと便利。

なお、NumPyにはパディングの関数が存在する。

[numpy.pad — NumPy v1.15 Manual](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.pad.html "numpy.pad — NumPy v1.15 Manual")


## 7. ミニバッチへの対応（アドバンス課題）
ここまでの課題はバッチサイズ1で良いとしてきた。  
しかし、実際は全結合層同様にミニバッチ学習が行われる。  
Conv1dクラスを複数のデータが同時に計算できるように変更する。

## 8. 任意のストライド数（アドバンス課題）
ストライドは1限定の実装をしてきたが、任意のストライド数に対応できるようにする。